## Accelerate Inference: Neural Network Pruning

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

from google.colab import files
print(tf.version.VERSION)

2.7.0


In [ ]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [ ]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [ ]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=64, epochs=300, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/300


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


352/352 [==============================] - 5s 12ms/step - loss: 1.0512 - accuracy: 0.5817 - val_loss: 0.9713 - val_accuracy: 0.6182
Epoch 2/300
352/352 [==============================] - 4s 12ms/step - loss: 0.9963 - accuracy: 0.6099 - val_loss: 0.9516 - val_accuracy: 0.6261
Epoch 3/300
352/352 [==============================] - 4s 11ms/step - loss: 0.9434 - accuracy: 0.6312 - val_loss: 1.0165 - val_accuracy: 0.6020
Epoch 4/300
352/352 [==============================] - 4s 11ms/step - loss: 0.9091 - accuracy: 0.6486 - val_loss: 0.9121 - val_accuracy: 0.6420
Epoch 5/300
352/352 [==============================] - 4s 12ms/step - loss: 0.8686 - accuracy: 0.6661 - val_loss: 0.9146 - val_accuracy: 0.6440
Epoch 6/300
352/352 [==============================] - 4s 11ms/step - loss: 0.8237 - accuracy: 0.6819 - val_loss: 0.8583 - val_accuracy: 0.6586
Epoch 7/300
352/352 [==============================] - 4s 12ms/step - loss: 0.8035 - accuracy: 0.6937 - val_loss: 0.7957 - val_accuracy: 0.6954
Epoc

In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 9ms/step - loss: 0.8278 - accuracy: 0.7545


In [ ]:
# perform pruning here

# get the weights 
weights = model.get_weights()

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# weights[7][:10]=0
model.set_weights(weights)


In [ ]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 7ms/step - loss: 1.0858 - accuracy: 0.5564


In [ ]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("uncompressed_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
files.download("uncompressed_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>